In [ ]:
lipidname = "OIPC"
head = "C P"
link = "G G"
tail = "CDCC CDDC"

membrane="test"
insane="./insane+SF.py"
mdparams="./test.mdp"
martinipath="./martini.ff"

# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}*

In [ ]:

print("Create itp")
!python {martinipath}/lipid-martini-itp-v06.py -o martini_v2_omaralab.itp -alname {lipidname} -alhead '{head}' -allink '{link}' -altail '{tail}'
print("Done")

In [ ]:
# build a simple membrane  to visualize this species
insane = !python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 1 -y 1 -z 16 -sol PW -center -charge 0 -u {lipidname}:1 -l {lipidname}:1

# use GMX to convert GRO file into a PDB
output = !gmx editconf -f {membrane}.gro -o  {membrane}.pdb

# create a file to automate rendering an image of the protein, 
# hide water molecules
# rotate display to view protein 
# display projection Orthographic
# setting "bead" style, 
# setting the background to white,
# turning of fthe XYZ axes,  
# and taking a snapshot 

with open("output.tcl", "w") as file:
    file.write("mol modselect 0 0 resid 1\n")
    file.write("rotate x by 120\n")
    file.write("rotate y by 90\n")
    file.write("translate by 0.0 1.0 0.0\n")
    file.write("display projection Orthographic\n")
    file.write("mol modstyle 0 0 VDW 1.000000 12.000000\n")
    file.write("axes location Off\n")
    file.write("color Display Background white\n")
    file.write("scale by 8\n")
    file.write("render snapshot image.tga {convert image.tga jpg:image.jpg; rm image.tga}\n")
    file.write("exit")
    
# run VMD to render the image 
output = !vmd {membrane}.pdb -e output.tcl 

# clean up temporary files
!rm output.tcl

import random
counter = random.randint(0,2e9)

from IPython.display import HTML, display
display(HTML('<h1>{}</h1><br/><img src="image.jpg?{}" alt="Image of {}" width="300">'.format(lipidname,counter,lipidname)))

In [ ]:
import os #Operating system specific commands
import re #Regular expression library

print("Test")
print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("Fatal error", line):
        success=False
    #if not success:
    print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)